# Beispiele zur geometrischen Nichtlinearität ebener Stabtragwerke

## Die dünne Blattfeder

Bei der dünnen Blattfeder handelt es sich um einen Stab, der auf einer Seite eingespannt ist und am freien Ende greift ein äußeres Moment an. Es handelt sich dabei um einen äußerst schlanken Stab, d.h. die Stablänge $l$ ist um ein Vielfaches größer als die Stabhöhe $h$ ($\rightarrow h \ll l$). Es treten hier auch große Verschiebungen und Verschiebungsableitungen auf, aber die Verzerrungen sind *klein*. An diesem Beispiel kann das Verhalten von Stabelementen getestet werden, da selbst für große Verschiebungen die analytische Lösung sich leicht bestimmen lässt.

![Blattfeder](Blattfeder2.png)

Wird die *verformte* Blattfeder an einer beliebigen Stelle durchtrennt, dann folgt aus den Gleichgewichtsbedingun, dass als Schnittgröße nur ein Biegemoment wirkt und dieses ist von der gleichen Größe wie das einwirkende äußere Moment und über die gesamte Länge konstant. Es wirkt somit weder eine Normal- noch eine Querkraft in der Blattfeder. Wenn die Normalkraft Null ist, muss auch die Stabdehnung $\overline{\varepsilon}$ über die gesamte Länge verschwinden. Wie bereits beim *membrane-locking* diskutiert, ist die Verformung der Blattfeder *kreisförmig* mit Radius $R$ und *dehnungslos*, d.h. es kommt zu keiner Längenänderung während der Verformung.

![Verhalten der Blattfeder](StrukturverhaltenBlattfeder.png)

Die Verschiebungen eines Punktes entlang der Blattfeder ergeben sich somit zu
$$
  u(x) = R\sin\left(\frac{x}{R}\right) - x  \qquad \mathrm{und} \qquad w(x) = R\left(\cos\left(\dfrac{x}{R}\right)-1\right) \; .
$$

Die bekannte Differentialgleichung der Biegelinie für *kleine* Verschiebungen, auch Momenten-Krümmungsbeziehung genannt, kann daher wie folgt modifiziert werden:
$$
\frac{w''}{(1+w'^2)^\frac{3}{2}} = -\frac{M_y}{EI_y}
$$

$$
\frac{1}{R} = \left|-\frac{M_y}{EI_y}\right| \quad \rightarrow \quad R = \frac{EI_y}{M}
$$

\begin{equation}
x = 2
\end{equation}